In [2]:
#upload file here
from google.colab import files
uploaded = files.upload()

Saving juneroute.csv to juneroute.csv


In [3]:
######### GRAPHS FOR USAGE OF EVERY VEHICLE PER DAY AND PER MONTH ###########
import pandas as pd
import folium as fs
from folium.features import DivIcon
pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings('ignore')
import altair as alt
import json
from datetime import datetime

#reading csv to panda dataframe
df = pd.read_csv('juneroute.csv')

#DROPPING ALL SAFE EVENTS TO GET UNSAFE EVENTS
df = df.sort_values(['Vehicle', 'Time'])



######THis function is to remove time from datetime string
####This is to help facilitate merging the dates together to be shown on graph
def rem_time(d):
    s = ''
    s = str(d.month) + '/' + str(d.day) + '/' + str(d.year)
    return s


####Reassigning all the values from 'Juneroute.csv' 'Time' column to only retain the date 
for ind in df.index:
  datetime_str  = df['Time'][ind]
  datetime_object = datetime.strptime(datetime_str, '%m/%d/%Y %H:%M')
  new=rem_time(datetime_object)
  df['Time'][ind] = new
  

###Sorting the values to help facilitate the plotting of graphs
df = df.sort_values(['Time','Vehicle'])


###Generating and appending graphs onto a list(to be used later) 
plot_group=[]
df_vehonly = df.drop_duplicates(['Vehicle'])
for i in df_vehonly['Vehicle']:
  chart = alt.Chart(df.loc[(df['Vehicle']) == i]).mark_bar().encode(
      y='Time',
      x='count(Event)',
      tooltip='count(Event)',
      color='Vehicle'
    ).properties(
        title="Event count per Day for vehicle "+'{:s}'.format(i),
        width=500,
        height=200
    )
  plot_group.append(chart)

#####DATA MASSAGING
##This block of codes is to get DISTINCT vehicle list
vehicle = []
df = df.sort_values(['Vehicle'])
df1 = df.drop_duplicates('Vehicle')
del df1['Event']
del df1['Time']
del df1['Driver']
del df1['Speed']
del df1['Address']
del df1['Longitude']
del df1['Latitude']
for i in df1['Vehicle']:           
  vehicle.append(i)

##This block of codes is to get event count for every distinct vehicle
avg = []
df1 = df1.sort_values(['Vehicle'])
df1 = df1.drop_duplicates('Vehicle')
for i in df1['Vehicle']:
  df_calc = df.loc[(df['Vehicle']) == i]
  df_calc = df_calc.sort_values(['Vehicle'])
  num = len(df_calc[df['Vehicle'] == i])
  avg.append(num)

##Merging the two lists onto a panda dataframe
veh_avg = pd.DataFrame()
veh_avg['Vehicle'] = vehicle
veh_avg['Event_count'] = avg


###Chart is the plot for the total number of events done per vehicle throughout the month
chart = alt.Chart(veh_avg).mark_bar().encode(
      x='Event_count',
      y='Vehicle',
      tooltip='Event_count',
      color='Vehicle'
    ).properties(
        title="Event count throughout the month for ALL",
        width=500,
        height=200
    )

##Appending chart to plot_group list and vertically concating the plots to display
plot_group.append(chart)
chart=alt.vconcat(*plot_group)

##Output chart
chart

alt.VConcatChart(...)

In [4]:
##FOLIUM MAP + ALTAIR CHARTS
import pandas as pd
import folium as fs
from folium.features import DivIcon
import altair as alt
import json




df = pd.read_csv('juneroute.csv')

df = df.sort_values(by=['Address'])

#DROPPING ALL UNSAFE EVENTS TO GET SAFE EVENTS
df = df.drop(df[df.Event == "Sudden brake in turn"].index)
df = df.drop(df[df.Event == "Braking while exiting turn"].index)
df = df.drop(df[df.Event == "Lane change"].index)
df = df.drop(df[df.Event == "Acceleration into turn"].index)

#Adding a column and values safe to the dataframe(to be used later)
df['event_type']="safe"

################################################################################
#This entire block of codes is to get distinct coordinate and address to display on map
#Coordinates and address are to reference SAFE events
#####DATA MASSAGING

#Creating a list for Latitude values
lat=[]
df_new = df.sort_values(by=['Latitude','Longitude'])
test = df.drop_duplicates(subset = 'Address')

del test['Time']
del test['Vehicle']
del test['Driver']
del test['Event']
del test['Speed']
del test['Address']
del test['Longitude']

for i in test['Latitude']:           
  lat.append(i)

#Creating a list for Longitude values
lon=[]
df_newnew = df.sort_values(by=['Latitude','Longitude'])
test1 = df.drop_duplicates(subset = 'Address')

del test1['Time']
del test1['Vehicle']
del test1['Driver']
del test1['Event']
del test1['Speed']
del test1['Address']
del test1['Latitude']

for i in test1['Longitude']:           
  lon.append(i)

#Creating a list for address values
adr=[]
df_newnewnew = df.sort_values(by=['Latitude','Longitude'])
test2 = df.drop_duplicates(subset = 'Address')

del test2['Time']
del test2['Vehicle']
del test2['Driver']
del test2['Event']
del test2['Speed']
del test2['Longitude']
del test2['Latitude']

for i in test2['Address']:           
  adr.append(i)

#Adding the previously created lists to a new dataframe called coord
#coord dataframe is used to help plot the markers of SAFE events on the map
coord = pd.DataFrame()
coord['latitude'] = lat
coord['longitude'] = lon
coord['address'] = adr
coord['event_type']="safe"

################################################################################
################################################################################

#Creating a seperate Dataframe to be used for UNSAFE Events
#####DATA MASSAGING
f1 = pd.read_csv('juneroute.csv')
df1 = f1.sort_values(by=['Address'])

#DROPPING ALL SAFE EVENTS TO GET UNSAFE EVENTS
df1 = df1.drop(df1[df1.Event == "Braking"].index)
df1 = df1.drop(df1[df1.Event == "Braking\xa0into turn"].index)
df1 = df1.drop(df1[df1.Event == "Acceleration"].index)
df1 = df1.drop(df1[df1.Event == "Turning"].index)
df1 = df1.drop(df1[df1.Event == "Braking while in turn"].index)

#Adding a column and values unsafe to the dataframe(to be used later)
df1['event_type']="unsafe"

################################################################################
#This entire block of codes is to get distinct coordinate and address to display on map
#Coordinates and address are to reference UNSAFE events

#Creating a list for Latitude values
lat1=[]
df_new1 = df1.sort_values(by=['Latitude','Longitude'])
test11 = df1.drop_duplicates(subset = 'Address')

del test11['Time']
del test11['Vehicle']
del test11['Driver']
del test11['Event']
del test11['Speed']
del test11['Address']
del test11['Longitude']

for i in test11['Latitude']:           
  lat1.append(i)

#Creating a list for Longitude values
lon1=[]
df_newnew1 = df1.sort_values(by=['Latitude','Longitude'])
test111 = df1.drop_duplicates(subset = 'Address')

del test111['Time']
del test111['Vehicle']
del test111['Driver']
del test111['Event']
del test111['Speed']
del test111['Address']
del test111['Latitude']

for i in test111['Longitude']:           
  lon1.append(i)

#Creating a list for Address values
adr1=[]
df_newnewnew1 = df1.sort_values(by=['Latitude','Longitude'])
test21 = df1.drop_duplicates(subset = 'Address')

del test21['Time']
del test21['Vehicle']
del test21['Driver']
del test21['Event']
del test21['Speed']
del test21['Longitude']
del test21['Latitude']

for i in test21['Address']:           
  adr1.append(i)


#Adding the previously created lists to a new dataframe called coord1
#coord dataframe is used to help plot the markers of UNSAFE events on the map
coord1 = pd.DataFrame()
coord1['latitude'] = lat1
coord1['longitude'] = lon1
coord1['address'] = adr1
coord1['event_type'] = "unsafe"

################################################################################
################################################################################

#Merging previously created dataframes into one
#coord_final will be used for plotting markers
#df_final will be used for popup data for driver performance
#df1 and df will be used for UNSAFE and SAFE popup data respectively
coord_final = pd.concat([coord,coord1])
df_final = pd.concat([df,df1])
df_final = df_final.sort_values(['Driver'])



################################################################################
################################################################################
#This block of codes is used to get driver's performance by percentage


#Creating a list for DISTINCT drivers
driver =[]
df_final = df_final.sort_values(['Driver'])
df_driversonly = df_final.drop_duplicates('Driver')
del df_driversonly['event_type']
for i in df_driversonly['Driver']:           
  driver.append(i)

#Getting the driver's performance by percentage for each driver(Formula used, (safe/(safe+unsafe))*100)
per =[]
df_final = df_final.sort_values(['Driver','event_type'])
for i in df_driversonly['Driver']:
  df_calc = df_final.loc[(df_final['Driver']) == i]
  df_calc = df_calc.sort_values(['event_type'])
  num =len(df_calc[df_calc['event_type'] == "unsafe"])
  num2=len(df_calc[df_calc['event_type'] == "safe"])
  avg_performance = round((num2/(num+num2) * 100),1)
  per.append(avg_performance)

#Adding lists created above onto a new dataframe
#To be used for plotting altair graph
df_performance = pd.DataFrame()
df_performance['Driver']=driver
df_performance['Performance']=per

################################################################################
################################################################################
#Creation of Folium Map

#Setting of icon style and color
#DivIcon will be used to display the markers on the map.
#Setting the style like this allows us to specify how we want the markers to look like
def number_DivIcon(color,number):
    """ Create a 'numbered' icon
    
    """
    icon = DivIcon(
            icon_size=(150,36),
            icon_anchor=(14,40),
#             html='<div style="font-size: 18pt; align:center, color : black">' + '{:02d}'.format(num+1) + '</div>',
            html="""<span class="fa-stack " style="font-size: 12pt" >>
                    <!-- The icon that will wrap the number -->
                    <span class="fa fa-circle-o fa-stack-2x" style="color : {:s}"></span>
                    <!-- a strong element with the custom content, in this case a number -->
                    <strong class="fa-stack-1x">
                         {:02d}  
                    </strong>
                </span>""".format(color,number)
        )
    return icon

#List of colors to choose from
#In our project we will only be using [0], [1] and [2]
col_hex = ['#FF0000',
 '#0000FF',
 '#000000',
 '#414487',
 '#39568c',
 '#31688e',
 '#2a788e',
 '#23888e',
 '#1f988b',
 '#22a884',
 '#35b779',
 '#54c568',
 '#7ad151',
 '#a5db36',
 '#d2e21b']


#Setting map location to singapore's coordinates
m = fs.Map(location=[1.290270,103.851959])

#Creating a title for our map
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Group 4 TLM2003</b></h3>
             '''
m.get_root().html.add_child(fs.Element(title_html))

#plotting markers onto map with filtered data
for grp_name,df_grp in coord_final.groupby('event_type'): #Looping through coord_final to plot marker locations
  feature_group=fs.FeatureGroup(grp_name)                 #Grouping by 'event_type' which is safe and unsafe
  for row in df_grp.itertuples():           
    loc = [row.latitude,row.longitude]                    #setting location of marker
    num = 0
    if row.event_type == "unsafe":                        #checking if event is unsafe. If unsafe, we will use df1 dataframe
      num = len(df1[df1['Address'] == row.address])
      chart = alt.Chart(df1.loc[(df1['Address']) == row.address]).mark_bar().encode( #plotting of bar chart
      y='Driver',
      x='count(Event)',
      tooltip='Event',
      color='Driver'
      ).properties(
      title="Unsafe event(s) at address "+'{:s}'.format(row.address),
      width=500,
      height=200
      )
      scatter = alt.Chart(df1.loc[(df1['Address']) == row.address]).mark_point().encode( #plotting scatter plot
      x='Speed',
      y='Driver',
      color='Driver',
      tooltip='Event'
      ).properties(
      title="Driver's speed",
      width=500,
      height=200
      )
      
      #vertically concating the charts to be displayed in popup
      plot = alt.vconcat(
      scatter, chart
      ).resolve_scale(
      color='independent'
      ).configure_view(
      stroke=None
      )
     
      c = col_hex[0] #Assiging color red to marker
      plot = json.loads(plot.to_json()) #converting the plot to json as we're using vegalite popup
      

    elif row.event_type == "safe": #For safe events, we will use df. The codes are the same with the exception of a different dataframe being used
      num = len(df[df['Address'] == row.address])
      chart = alt.Chart(df.loc[(df['Address']) == row.address]).mark_bar().encode(
      y='Driver',
      x='count(Event)',
      tooltip='Event',
      color='Driver'
      ).properties(
      title="Safe event(s) at address "+'{:s}'.format(row.address),
      width=500,
      height=200
      )
      scatter = alt.Chart(df.loc[(df['Address']) == row.address]).mark_point().encode(
      x='Speed',
      y='Driver',
      color='Driver',
      tooltip='Event'
      ).properties(
      title="Driver's speed",
      width=500,
      height=200
      )

      plot = alt.vconcat(
      scatter, chart
      ).resolve_scale(
      color='independent'
      ).configure_view(
      stroke=None
      )

      c=col_hex[1] #Color set here is blue
      plot = json.loads(plot.to_json())

    #Setting popup properties
    popup = fs.Popup(max_width=650)
    fs.features.VegaLite(plot,height=400, width=800).add_to(popup) #Adding plot to popup via vegalite
    #Setting marker properties and adding each marker to feature_group
    fs.Marker(
      location=loc, popup=popup,
      icon= number_DivIcon(c,0+num),
      markerColor=c,tooltip='<strong>Click here to see Popup</strong>'
    ).add_to(feature_group)
  feature_group.add_to(m) #Doing this allows us to show safe and unsafe events separately 
fs.LayerControl().add_to(m) #enabling layer control


#Setting marker to display Driver's performance
loc = [1.313041,103.647797]

##DRIVER PERFORMANCE % 
performance_graph = alt.Chart(df_performance).mark_bar().encode(
      y='Performance',
      x='Driver',
      color='Driver',
      tooltip='Performance'
      ).properties(
      title="Performance %",
      width=150,
      height=400
      )

##TOTAL UNSAFE EVENTS PER DRIVER
total_graph = alt.Chart(df1).mark_bar().encode(
      y='count(event_type)',
      x='Driver',
      color='Driver',
      tooltip='count(event_type)'
      ).properties(
      title="Unsafe events per driver",
      width=150,
      height=400
      )

##TOTAL SAFE EVENTS PER DRIVER
total_graph1 = alt.Chart(df).mark_bar().encode(
      y='count(event_type)',
      x='Driver',
      color='Driver',
      tooltip='count(event_type)'
      ).properties(
      title="Safe events per driver",
      width=150,
      height=400
      )

##horizontally concating the graphs
plot1 = alt.hconcat(
      total_graph, total_graph1 ,performance_graph
      ).resolve_scale(
      color='independent'
      ).configure_view(
      stroke=None
      )

#Setting popup and marker properties for plot1
plot1 = json.loads(plot1.to_json())
popup = fs.Popup(max_width=900)
fs.features.VegaLite(plot1,height=400, width=900).add_to(popup)

fs.Marker(
   location=loc, popup=popup,
      icon= number_DivIcon(col_hex[2],00),
      markerColor=c,tooltip='<strong>Click here to see Drivers performance</strong>'
    ).add_to(m)

       
#display map       
m